In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import os
import collections
import itertools

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ssl

In [4]:
from six.moves import urllib

In [ ]:
URL_TRAIN_PATH = 'https://download.tensorflow.org/data/iris_training.csv'
URL_TEST_PATH = 'https://download.tensorflow.org/data/iris_test.csv'

DOWNLOADED_FILENAME_TRAIN = 'iris_training.csv'
DOWNLOADED_FILENAME_TEST = 'iris_test.csv'

def download_data():
  if not os.path.exists(DOWNLOADED_FILENAME_TRAIN):
    filename, _ = urllib.request.urlretrieve(URL_TRAIN_PATH, DOWNLOADED_FILENAME_TRAIN)
  print('Found and verified file from this path: ', URL_TRAIN_PATH)
  print('Downloaded file: ', DOWNLOADED_FILENAME_TRAIN)
  
  if not os.path.exists(DOWNLOADED_FILENAME_TEST):
    filename, _ = urllib.request.urlretrieve(URL_TEST_PATH, DOWNLOADED_FILENAME_TEST)
  print('Found and verified file from this path: ', URL_TEST_PATH)
  print('Downloaded file: ', DOWNLOADED_FILENAME_TEST)

download_data()

In [8]:
FEATURE_NAMES = [
  'SepalLengthCm',
  'SepalWidthCm',
  'PetalLengthCm',
  'SepalWidthCm'
]

In [9]:
def parse_csv(line):
  parsed_line = tf.decode_csv(line, [[0.], [0.], [0.], [0.], [0]])
  labels = parsed_line[-1:]
  del parsed_line[-1]
  features = dict(zip(FEATURE_NAMES, parsed_line))
  return features, labels

In [12]:
def get_features_labels(filename, shuffle=False, repeat_count=1):
  dataset = (tf.data.TextLineDataset(filename).skip(1).map(parse_csv))
  if shuffle:
    dataset = dataset.shuffle(buffer_size=256)
  dataset = dataset.repeat(repeat_count)
  dataset = dataset.batch(32)
  iterator = dataset.make_one_shot_iterator()
  batch_features, batch_labels = iterator.get_next()
  return batch_features, batch_labels

In [ ]:
batch_features, batch_labels = get_features_labels(DOWNLOADED_FILENAME_TRAIN)

In [14]:
feature_columns = [tf.feature_column.numeric_column(k) for k in FEATURE_NAMES]

In [16]:
classifier_model = tf.estimator.DNNClassifier(
  feature_columns = feature_columns,
  hidden_units = [10,10],
  n_classes = 3
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0be15544d0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/tmpIeVwTl', '_save_summary_steps': 100}


In [ ]:
classifier_model.train(
  input_fn = lambda: get_features_labels(DOWNLOADED_FILENAME_TRAIN, shuffle=True, repeat_count=20)
)

In [17]:
results = classifier_model.evaluate(
  input_fn = lambda: get_features_labels(DOWNLOADED_FILENAME_TEST, shuffle=False, repeat_count=4)
)